In [1]:
import os
import sys
from typing import List, Dict

import lightgbm as lgb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
import plotly.express as px

# Code 경로 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

c:\Users\com\Documents\GitHub\level1-classificationinmachinelearning-recsys-06


In [2]:
# 파일 호출
data_path: str = "..\..\data"
## raw.csv가 없는 경우 실행
# from Code.dataset.merge_all import merge_all
# df = merge_all(data_path)
data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "raw.csv"))
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))  # ID, target 열만 가진 데이터 미리 호출

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\com\AppData\Local\Temp\ipykernel_7140\3026538581.py:2: SyntaxWarning: invalid escape sequence '\.'
  data_path: str = "..\..\data"


In [ ]:
columns = data.columns.tolist()
columns

In [3]:
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "premium_gap",
    'hourly_market-data_coinbase-premium-index_coinbase_premium_index': "premium_index",
    'hourly_market-data_funding-rates_all_exchange_funding_rates': "funding_rates",
    'hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations': "long_liquidations",
    'hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations': "short_liquidations",
    'hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd': "long_liquidations_usd",
    'hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd': "short_liquidations_usd",
    'hourly_market-data_open-interest_all_exchange_all_symbol_open_interest': "all_symbol_open_interest",
    'hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close': "close",
    'hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_volume':'volume',
    'hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume': 'buy_volume',
    'hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume':'sell_volume',
    'hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio':'buy_ratio',
    'hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio':'sell_ratio',
    'hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio': 'buy_sell_ratio',
    'hourly_network-data_addresses-count_addresses_count_active':'count_active',
    'hourly_network-data_addresses-count_addresses_count_sender':'count_sender',
    'hourly_network-data_addresses-count_addresses_count_receiver':'count_receiver',
    'hourly_network-data_block-bytes_block_bytes':'block_bytes',
    'hourly_network-data_block-count_block_count':'block_count',
    'hourly_network-data_block-interval_block_interval':'block_interval',
    'hourly_network-data_blockreward_blockreward':'blockreward',
    'hourly_network-data_blockreward_blockreward_usd':'blockreward_usd',
    'hourly_network-data_difficulty_difficulty':'difficulty',
    'hourly_network-data_fees-transaction_fees_transaction_mean':'transaction_mean',
    'hourly_network-data_fees-transaction_fees_transaction_mean_usd':'transaction_mean_usd',
    'hourly_network-data_fees-transaction_fees_transaction_median':'transaction_median',
    'hourly_network-data_fees-transaction_fees_transaction_median_usd':'transaction_median_usd',
    'hourly_network-data_fees_fees_block_mean':'block_mean',
    'hourly_network-data_fees_fees_block_mean_usd':'block_mean_usd',
    'hourly_network-data_fees_fees_total':'fees_total',
    'hourly_network-data_fees_fees_total_usd':"fees_total_usd",
    'hourly_network-data_fees_fees_reward_percent':'fees_reward_percent',
    'hourly_network-data_hashrate_hashrate':'hashrate',
    'hourly_network-data_supply_supply_total':'supply_total',
    'hourly_network-data_supply_supply_new':'supply_new',
    'hourly_network-data_tokens-transferred_tokens_transferred_total': 'tokens_transferred_total',
    'hourly_network-data_tokens-transferred_tokens_transferred_mean':'tokens_transferred_mean',
    'hourly_network-data_tokens-transferred_tokens_transferred_median':'tokens_transferred_median',
    'hourly_network-data_transactions-count_transactions_count_total':'transactions_count_total',
    'hourly_network-data_transactions-count_transactions_count_mean':'transactions_count_mean',
    'hourly_network-data_utxo-count_utxo_count':'utxo_count',
    'hourly_network-data_velocity_velocity_supply_total':'velocity_supply_total'

}
df = data[cols_dict.keys()].rename(cols_dict, axis=1)

In [76]:
df.to_csv('features_select_df.csv',index=False)

In [ ]:
df.isnull().sum()

In [4]:
# 시차(Lag) 특성을 생성할 컬럼 리스트 정의
columns_to_lag = [
    'premium_gap',
    'premium_index',
    'funding_rates',
    'all_symbol_open_interest',
    'close',
    'volume',
]

In [6]:
# 시차 (Lag) 특성 생성 함수 정의
def create_lag_features(df, columns, max_lag):
    df = df.copy()
    for col in columns:
        for lag in range(1, max_lag + 1):
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    return df

In [9]:
#from preprocessor import PreProcessor
print(os.getcwd())
sys.path.append('..\Code\pre_procecss')
#from preprocessor import PreProcessor
#preprocessor = PreProcessor(df, config_path='config-sample.yaml')

c:\Users\com\Documents\GitHub\level1-classificationinmachinelearning-recsys-06\EDA\백우성


<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\com\AppData\Local\Temp\ipykernel_7140\3668511730.py:3: SyntaxWarning: invalid escape sequence '\C'
  sys.path.append('..\Code\pre_procecss')


In [ ]:
# 최대 시차 생성
max_lag = 3

# 시차 특성 생성
data_with_lags = create_lag_features(df, columns_to_lag, max_lag)

data_with_lags

In [8]:
# 이동 평균 (Moving Average) 특성 생성 함수 정의
def create_moving_average_features(df, columns, windows):
    df = df.copy()
    for col in columns:
        for window in windows:
            df[f'{col}_ma_{window}'] = df[col].rolling(window=window).mean()
    return df

In [9]:
# 이동 평균을 계산할 윈도우 크기 설정
windows = [3,6,12]

# 이동 평균 특성 생성
data_with_moving_average = create_moving_average_features(data_with_lags, columns_to_lag, windows)


In [10]:
# 추가적인 시계열 특성 생성 함수 정의
def create_additional_ts_features(df, columns, windows):
    df = df.copy()
    for col in columns:
        for window in windows:
            # 이동 평균 편차 (Volatility)
            df[f'{col}_std_{window}'] = df[col].rolling(window=window).std()
            # 차분 (Difference)
            df[f'{col}_diff_{window}'] = df[col] - df[col].shift(window)
            # 지수 이동 평균 (EMA)
            df[f'{col}_ema_{window}'] = df[col].ewm(span=window, adjust=False).mean()
    return df

In [11]:
# 결측값 제거 함수 (시계열 특성에서 발생한 결측값만 제거)
def remove_ts_related_missing_values(df, columns, max_lag):
    # 시차(Lag) 및 시계열 특성으로 인한 결측값을 처리할 컬럼 정의
    lagged_columns = [f'{col}_lag_{lag}' for col in columns for lag in range(1, max_lag + 1)]
    moving_avg_columns = [f'{col}_ma_{window}' for col in columns for window in [3, 6, 12]]
    volatility_columns = [f'{col}_std_{window}' for col in columns for window in [3, 6, 12]]
    
    # 해당 컬럼들에서 발생한 결측값만 제거
    relevant_columns = lagged_columns + moving_avg_columns + volatility_columns
    df_cleaned = df.dropna(subset=relevant_columns).reset_index(drop=True)
    
    return df_cleaned

In [12]:
# 추가적인 시계열 특성 생성
data_with_ts_features = create_additional_ts_features(data_with_moving_average, columns_to_lag, windows)

# 최종 데이터셋에서 결측값 제거 (시계열 특성 생성으로 인한 결측값)
data_final = remove_ts_related_missing_values(data_with_ts_features,columns_to_lag,max_lag)
#data_final

In [13]:
_target = data_with_ts_features["target"]

# _type이 train인 데이터의 결측치만 삭제
df1 = remove_ts_related_missing_values(data_with_ts_features[data_with_ts_features['_type']=='train'],columns_to_lag,max_lag)
# _type이 test인 데이터는 결측치를 삭제하지 않고 그대로 유지
df2 = data_with_ts_features[data_with_ts_features['_type']=='test'].ffill().assign(target=_target)

# train과 test 데이터를 다시 합침
final_data = pd.concat([df1, df2], axis=0).reset_index(drop=True)

# 결과 확인
print(f"train 데이터: {len(df1)}, test 데이터: {len(df2)}, 최종 데이터: {len(final_data)}")


train 데이터: 8722, test 데이터: 2792, 최종 데이터: 11514


In [101]:
final_data['target'].value_counts()

target
2.0    3652
1.0    3531
3.0     804
0.0     735
Name: count, dtype: int64

In [72]:
# XGBoost 모델 훈련 및 평가를 위해 데이터 분할
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

# _type에 따라 train, test 분리
train_df = final_data.loc[final_data["_type"] == "train"].drop(columns=["_type"])
test_df = final_data.loc[final_data["_type"] == "test"].drop(columns=["_type"])

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1),
    train_df["target"].astype(int),
    test_size=0.2,
    random_state=42,
)

# XGBoost 모델 생성 및 학습
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    learning_rate = 0.05,
    max_depth=5,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42)

# XGBoost 모델 훈련
xgb_model.fit(x_train, y_train)

# 예측
y_train_pred = xgb_model.predict(x_train)
y_valid_pred = xgb_model.predict(x_valid)

# 성능 평가 (훈련)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_classification_rep = classification_report(y_train, y_train_pred)

# 성능 평가 (검증)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
valid_classification_rep = classification_report(y_valid, y_valid_pred)

print(f"훈련 정확도: {train_accuracy}")
print(train_classification_rep)
print(f"검증 정확도: {valid_accuracy}")
print(valid_classification_rep)

# 교차 검증
scores = cross_val_score(xgb_model, x_train, y_train, cv=5, scoring='accuracy')
print(f"교차 검증 평균 정확도: {scores.mean()}")

훈련 정확도: 0.7279633080120396
              precision    recall  f1-score   support

           0       0.96      0.29      0.45       594
           1       0.73      0.79      0.76      2809
           2       0.70      0.84      0.76      2932
           3       0.95      0.34      0.50       642

    accuracy                           0.73      6977
   macro avg       0.83      0.57      0.62      6977
weighted avg       0.76      0.73      0.71      6977

검증 정확도: 0.4681948424068768
              precision    recall  f1-score   support

           0       0.71      0.12      0.21       141
           1       0.47      0.51      0.49       722
           2       0.47      0.58      0.52       720
           3       0.26      0.07      0.12       162

    accuracy                           0.47      1745
   macro avg       0.48      0.32      0.33      1745
weighted avg       0.47      0.47      0.44      1745

교차 검증 평균 정확도: 0.4585053044540982


In [14]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, SVMSMOTE

def get_over_sampler(x_train, y_train, strategy='ADASYN', sampling_strategy='auto'):
    if strategy == 'SMOTE':
        sampler = SMOTE(sampling_strategy=sampling_strategy)
    elif strategy == 'BorderlineSMOTE':
        sampler = BorderlineSMOTE(sampling_strategy=sampling_strategy)
    elif strategy == 'ADASYN':
        sampler = ADASYN(sampling_strategy=sampling_strategy)
    elif strategy == 'SVMSMOTE':
        sampler = SVMSMOTE(sampling_strategy=sampling_strategy)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    x_resampled, y_resampled = sampler.fit_resample(x_train, y_train)
    return x_resampled, y_resampled

In [33]:
# XGBoost 모델 훈련 및 평가를 위해 데이터 분할
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

# _type에 따라 train, test 분리
train_df = final_data.loc[final_data["_type"] == "train"].drop(columns=["_type"])
train_df = train_df.ffill()
test_df = final_data.loc[final_data["_type"] == "test"].drop(columns=["_type"])

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1),
    train_df["target"].astype(int),
    test_size=0.2,
    random_state=42,
)
sampling_strategy = {0:1500, 1: 3500, 2:3500, 3:1500}
smote = SMOTE(sampling_strategy=sampling_strategy,random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# XGBoost 모델 생성 및 학습
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    learning_rate = 0.05,
    max_depth=5,
    n_estimators=500,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42)

# XGBoost 모델 훈련
xgb_model.fit(x_train_resampled, y_train_resampled)

# 예측
y_train_pred = xgb_model.predict(x_train_resampled)
y_valid_pred = xgb_model.predict(x_valid)

# 성능 평가 (훈련)
train_accuracy = accuracy_score(y_train_resampled, y_train_pred)
train_classification_rep = classification_report(y_train_resampled, y_train_pred)

# 성능 평가 (검증)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
valid_classification_rep = classification_report(y_valid, y_valid_pred)

print(f"훈련 정확도: {train_accuracy}")
print(train_classification_rep)
print(f"검증 정확도: {valid_accuracy}")
print(valid_classification_rep)

# 교차 검증
scores = cross_val_score(xgb_model, x_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
print(f"교차 검증 평균 정확도: {scores.mean()}")

훈련 정확도: 0.9843
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1500
           1       0.98      0.99      0.99      3500
           2       0.98      0.99      0.98      3500
           3       0.99      0.97      0.98      1500

    accuracy                           0.98     10000
   macro avg       0.99      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

검증 정확도: 0.439541547277937
              precision    recall  f1-score   support

           0       0.22      0.13      0.17       141
           1       0.47      0.51      0.49       722
           2       0.46      0.49      0.48       720
           3       0.24      0.16      0.19       162

    accuracy                           0.44      1745
   macro avg       0.35      0.32      0.33      1745
weighted avg       0.42      0.44      0.43      1745

교차 검증 평균 정확도: 0.6012000000000001


In [34]:
print(y_train_resampled.value_counts())

target
1    3500
2    3500
0    1500
3    1500
Name: count, dtype: int64


In [35]:
y_test_pred = xgb_model.predict(test_df.drop(["target", "ID"],axis=1))

In [36]:
submission_df = submission_df.assign(target=pd.DataFrame(y_test_pred))
submission_df

,ID,target
0,2024-01-01 00:00:00,1
1,2024-01-01 01:00:00,1
2,2024-01-01 02:00:00,1
3,2024-01-01 03:00:00,2
4,2024-01-01 04:00:00,1
...,...,...
2787,2024-04-26 03:00:00,0
2788,2024-04-26 04:00:00,0
2789,2024-04-26 05:00:00,1
2790,2024-04-26 06:00:00,0


In [38]:
submission_df['target'].value_counts()

target
1    964
2    913
0    805
3    110
Name: count, dtype: int64

In [39]:
submission_df.to_csv("xgb_test3_lag.csv",index=False)